In [1]:
import torch
import utils
import TextNet
import TeacherNet
import ImageNet
import torch.optim as optim
import time
import argparse
import numpy as np
import sys
import queue as queue_lib
import easydict
from embeddings import build_vocab, build_embed, build_train, replace_values

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from knockknock import slack_sender

from torch.utils.tensorboard import SummaryWriter

Torch Version:  1.3.0+cu92


In [2]:
def momentum_update(model_q, model_k, beta=0.999):
    param_k = model_k.state_dict()
    param_q = model_q.named_parameters()
    for n, q in param_q:
        param_k[n].data.copy_(beta*param_k[n].data + (1-beta)*q.data)
    model_k.load_state_dict(param_k)


In [5]:
def main():
#     LOGGER = utils.Logger()
#     PARSER = argparse.ArgumentParser()
#     PARSER.add_argument("--epochs", help="number of epochs", default=100, type=int)
#     PARSER.add_argument("--batchsize", help="batch size", default=32, type=int)
#     PARSER.add_argument("--train_modality_net", help="whether to train modality-specific network", default=0, type=int)
#     PARSER.add_argument("--loss_function", help="which loss function", default=1, type=int)
#     PARSER.add_argument("--verbose", help="print information", default=1, type=int)


#     MY_ARGS = PARSER.parse_args()

#     LOGGER.info("=============================================================")
#     print(MY_ARGS)
#     LOGGER.info("=============================================================")
    MY_ARGS = easydict.EasyDict({
            "batchsize": 32,
            "epochs": 100,
            "train_modality_net": 0,
            "loss_function": 1,
            "verbose":1,
            "arch":1
    })

    writer = SummaryWriter('logs')
    
    train_img = torch.load("../cached_data/train_img")
    train_cap = torch.load("../cached_data/train_cap")
    train_mask = torch.load("../cached_data/train_mask")

    val_img = torch.load("../cached_data/val_img")
    val_cap = torch.load("../cached_data/val_cap")
    val_mask = torch.load("../cached_data/val_mask")

    print("Loaded train data", train_img.size(), train_cap.size(), train_mask.size())
    print("Loaded val data", val_img.size(), val_cap.size(), val_mask.size())

    MOMENT = 0.9
    BATCH_SIZE = MY_ARGS.batchsize
    NB_EPOCHS = MY_ARGS.epochs
    device = "cuda: 1"

    vocab = build_vocab(train_cap) # all unique words
#     print(vocab[0])

    embedding_matrix = build_embed(vocab) # embeddings gloves
#     print(np.shape(embedding_matrix))

    train_cap = torch.load("../cached_data/cap_replace")
    
    train_data = TensorDataset(train_img, train_cap, train_mask)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE, num_workers=2)
    valid_data = TensorDataset(val_img, val_cap, val_mask)
    valid_sampler = RandomSampler(valid_data)
    valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=BATCH_SIZE, num_workers=2)

    # define Text Network    
    text_net = TextNet.Text_Net(embedding_matrix )
    text_net.to(device)

    # define Image Network
    model_name = "alexnet"
    feature_extract = True
    vision_net = ImageNet.Image_Net()
    vision_net = vision_net.initialize_model(model_name, feature_extract, use_pretrained=True)
    vision_net.to(device)
    
    # define Teacher Network
    teacher_net1 = TeacherNet.Teacher_Net()
    teacher_net2 = TeacherNet.Teacher_Net()
    
    ranking_loss = TeacherNet.ContrastiveLoss(1, device)
    
    teacher_net1.to(device)
    teacher_net2.to(device)
    ranking_loss.to(device)

    param_names = teacher_net1.state_dict().keys()

    # optimizer
    optimizer = optim.Adam(teacher_net1.parameters(), lr=3e-5)

    
    print("Start to train")
    start_time = time.time()
    train_losses = []
    train_accs = []
    val_losses = []
    val_accs = []
    NEG_SAMPLES = TeacherNet.CustomedQueue()
    QUEUE_SIZE = 64

    for epoch in range(NB_EPOCHS):
        """
        Training
        """
        running_loss = []
        running_corrects = 0.0
        total_samples = 0

        for step, batch in enumerate(train_dataloader):
            img, cap, mask = tuple(t.to(device) for t in batch)
            if NEG_SAMPLES.empty():
                with torch.set_grad_enabled(False):
                    txt_vec = teacher_net2.forward(text_net.forward(cap))
                    NEG_SAMPLES.enqueue(txt_vec)
                continue

            else:
                with torch.set_grad_enabled(False):
                    img_feature = vision_net.forward(img)
                    txt_feature = text_net.forward(cap)
                   

                with torch.set_grad_enabled(True):
                    img_vec = teacher_net1.forward(img_feature)
                    txt_vec = teacher_net2.forward(txt_feature)
                    neg_txt_vec = NEG_SAMPLES.get_tensor()
                    neg_txt_vec = neg_txt_vec.detach()
                    txt_vec = txt_vec.detach()

                    # print("img", img_vec)
                    loss = ranking_loss(img_vec, txt_vec, neg_txt_vec)
                    running_loss.append(loss.item())
                    loss.backward()

                    torch.nn.utils.clip_grad_norm_(parameters=teacher_net1.parameters(),
                                                   max_norm=1.0)
                    # for param in teacher_net1.parameters():
                    #     print(param.grad.data.sum())

                    optimizer.step()
                    optimizer.zero_grad()
                    # print()

                with torch.set_grad_enabled(False):
                    img_vec = teacher_net1.forward(img_feature)
                    txt_vec = teacher_net2.forward(txt_feature)
                    preds = ranking_loss.predict(img_vec, txt_vec)
                    NEG_SAMPLES.enqueue(txt_vec)

                for key in param_names:
                    teacher_net2.state_dict()[key] = teacher_net2.state_dict()[key] * MOMENT + \
                                                     (1 - MOMENT) * teacher_net1.state_dict()[key]

                running_corrects += sum([(i == preds[i]) for i in range(len(preds))])
                total_samples += len(preds)           
            
            if NEG_SAMPLES.size >= QUEUE_SIZE:
                NEG_SAMPLES.dequeue(BATCH_SIZE)

#         LOGGER.info("Epoch %d: train loss = %f, max=%f min=%f" % (epoch, np.average(running_loss),
#                                                                   np.max(running_loss),
#                                                                   np.min(running_loss)))
#         LOGGER.info(
#             "          train acc = %f (%d/%d)" % (
#                 float(running_corrects / total_samples), running_corrects, total_samples))
        
        print("Epoch %d: train loss = %f" % (epoch, np.average(running_loss)))
        print(("train acc = %f (%d/%d)" % (float(running_corrects/total_samples), 
                                           running_corrects, total_samples)))

        train_losses.append(np.average(running_loss))
        train_accs.append(float(running_corrects / total_samples))
        
        writer.add_graph(vision_net,img)
        writer.add_scalar('Loss/train', np.average(running_loss), epoch)
        writer.close()

        
#         model_name = "%d" % running_corrects
    #     torch.save(teacher_net1.state_dict(), "models/enc1-%s-norm" % model_name)
    #     torch.save(teacher_net2.state_dict(), "models/enc2-%s-norm" % model_name)

#         print(train_losses)
#         print(train_accs)
#         print()

#         """
#         Validating
#         """

#         running_loss = []
#         running_corrects = 0.0
#         total_samples = 0
#         teacher_net1.eval()
#         VAL_NEG_SAMPLES = TeacherNet.CustomedQueue()
#         with torch.no_grad():
#             for step, batch in enumerate(valid_dataloader):
#                 img, cap, mask = tuple(t.to(device) for t in batch)
#                 if VAL_NEG_SAMPLES.empty():
#                     txt_vec = teacher_net2.forward(text_net.forward(cap))
#                     VAL_NEG_SAMPLES.enqueue(txt_vec)
#                     continue

#                 else:
#                     img_vec = teacher_net1.forward(vision_net.forward(img))
#                     txt_vec = teacher_net2.forward(text_net.forward(cap))
#                     neg_txt_vec = VAL_NEG_SAMPLES.get_tensor()

#     #                     print(img_vec)
#     #                     print(txt_vec)
#     #                     print(VAL_NEG_SAMPLES.get_tensor(False))
#     #                     print()

#                     loss = ranking_loss(img_vec, txt_vec, neg_txt_vec)
#                     running_loss.append(loss.item())
#                     _, preds = ranking_loss.return_logits(img_vec, txt_vec, neg_txt_vec)
#                     VAL_NEG_SAMPLES.enqueue(txt_vec)
#                     VAL_NEG_SAMPLES.dequeue(BATCH_SIZE)
#                     running_corrects += sum([(0 == preds[i]) for i in range(len(preds))])
#                     total_samples += len(preds)



#         LOGGER.info("          val loss = %f, max=%f min=%f" % (np.average(running_loss),
#                                                                 np.max(running_loss),
#                                                                 np.min(running_loss)))
#         LOGGER.info(
#             "          val acc = %f (%d/%d)" % (
#                 float(running_corrects / total_samples), running_corrects, total_samples))

#         val_losses.append(np.average(running_loss))
#         val_accs.append(float(running_corrects / total_samples))

#     model_name = "%d" % running_corrects
#     # torch.save(teacher_net1.state_dict(), "models/enc1-%s-norm" % model_name)
#     # torch.save(teacher_net2.state_dict(), "models/enc2-%s-norm" % model_name)

#     print(train_losses)
#     print(train_accs)
#     print(val_losses)
#     print(val_accs)
#     print()


In [ ]:
if __name__ == '__main__':
    main()

Loaded train data torch.Size([10000, 3, 224, 224]) torch.Size([10000, 52]) torch.Size([10000, 52])
Loaded val data torch.Size([5000, 3, 224, 224]) torch.Size([5000, 43]) torch.Size([5000, 43])
4096
Start to train
Epoch 0: train loss = 3.496506
train acc = 0.030999 (309/9968)
Epoch 1: train loss = 3.891821
train acc = 0.029600 (296/10000)
Epoch 2: train loss = 3.496506
train acc = 0.032100 (321/10000)
Epoch 3: train loss = 3.891821
train acc = 0.030900 (309/10000)
Epoch 4: train loss = 3.496506
train acc = 0.028800 (288/10000)
